In [ ]:
import time
from src.HelperFunctions.helper import *
from src.LineSegmentation.lineSegment import *
from src.LineSegmentation.lineSegmentAStar import *
from src.SlantNormalization.slantNormalize import *
from characters import Character_Classification
from classifier import Classifier

In [ ]:
SAVEPICTURES = True # set to false if you do not want to save pictures in between steps

img = getImage("images/05.jpg")
rotatedImage, slope = findSlope(img, 10, 1)
print("Best angle: ", slope)

In [ ]:
images = lineSegmentAStar(rotatedImage)
print("segmenting lines")
for i in range(0,len(images)):
    images[i] = np.transpose(images[i])
    if SAVEPICTURES: cv.imwrite("line_" + str(i) + ".bmp", (1 - images[i]) * 255)
    if SAVEPICTURES: print("saving picture ", i)

In [ ]:
print("doing slant normalization")
slantAngles = []
for i in range(0,len(images)):
    slantAngle, dst = slantNormalize(images[i])
    if SAVEPICTURES: cv.imwrite("line_" + str(i) + "_deslanted.bmp", (1 - dst) * 255)
    images[i] = dst
    slantAngles.append(slantAngle)

In [ ]:
print("doing character classification and segmentation")
segment_size = (30,30)
window_size = (30*6, 30*3)
model_path = './trained_models/model_dimension3_500_epochs.pt'
cc = Character_Classification(segment_size, window_size, model_path)
imgs = []
labels = []
cnt = 0
for image in images:
    print(cnt)
    cnt +=1
    img, lab = cc.run_classification(image)
    imgs.extend(img)
    labels.extend(lab)

In [ ]:
print(len(imgs))
print(len(labels))

print(imgs[90])
cv.imshow("Window",images[4])

In [ ]:
print("doint style classification")
styleClassifier = Classifier("char_num_acc_lda_k3.txt", 3)
style = styleClassifier.classifyList(imgs, labels)
print(style)
